# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [9]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/Users/sadhanadesai/Desktop/Udemy-ML,DS,DL/MLCourse/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/Users/sadhanadesai/Desktop/Udemy-ML,DS,DL/MLCourse/emails/ham', 'ham'))


/Users/sadhanadesai/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Let's have a look at that DataFrame:

In [10]:
data.head()

,class,message
"/Users/sadhanadesai/Desktop/Udemy-ML,DS,DL/MLCourse/emails/spam/00249.5f45607c1bffe89f60ba1ec9f878039a",spam,"Dear Homeowner,\n\n \n\nInterest Rates are at ..."
"/Users/sadhanadesai/Desktop/Udemy-ML,DS,DL/MLCourse/emails/spam/00373.ebe8670ac56b04125c25100a36ab0510",spam,ATTENTION: This is a MUST for ALL Computer Use...
"/Users/sadhanadesai/Desktop/Udemy-ML,DS,DL/MLCourse/emails/spam/00214.1367039e50dc6b7adb0f2aa8aba83216",spam,This is a multi-part message in MIME format.\n...
"/Users/sadhanadesai/Desktop/Udemy-ML,DS,DL/MLCourse/emails/spam/00210.050ffd105bd4e006771ee63cabc59978",spam,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...
"/Users/sadhanadesai/Desktop/Udemy-ML,DS,DL/MLCourse/emails/spam/00033.9babb58d9298daa2963d4f514193d7d6",spam,This is the bottom line. If you can GIVE AWAY...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [11]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [27]:
examples = ['Free Viagra now!!!', 'Hi Bob, how about a game of golf tomorrow?']
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham', 'spam'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [28]:
from sklearn.model_selection import train_test_split

trainDf, testDf = train_test_split(data, test_size=0.2)
trainVect = CountVectorizer()
trainCounts = vectorizer.fit_transform(trainDf['message'].values)
trainTargets = trainDf['class'].values
classifier.fit(trainCounts, trainTargets)

testCounts = vectorizer.transform(testDf['message'])
testPredictions = classifier.predict(testCounts)

correct = 0
total = 0

for index, (value1, value2) in enumerate(zip(testPredictions, testDf['class'])):
    if value1 == value2:
        correct += 1
    total += 1

percentCorrect = correct/total
print(f'Percent correct: {percentCorrect*100}%')

Percent correct: 93.83333333333333%
